In [138]:
import numpy as np
import pandas as pd
import sys

from arrow import Arrow
from bcpp_community import communities
from bcpp_status.models import StatusHistory
from bcpp_subject.models import SubjectVisit, SubjectConsent, HivCareAdherence
from datetime import datetime
from django.db import connection
from edc_constants.constants import YES, NO, NEG, UNK
from edc_pdutils.model_to_dataframe import ModelToDataframe
from pprint import pprint
from math import floor
from bcpp_subject.admin import HivCareAdherenceAdmin
from edc_base.model_mixins.constants import DEFAULT_BASE_FIELDS
from IPython.display import Markdown, display


In [139]:
def printmd(string):
    display(Markdown(string))
    

In [140]:
class FormDescriber:

    def __init__(self, admin_cls=None):
        self.lines = []
        self.admin_cls = admin_cls
        self.model_cls = admin_cls.form._meta.model
        self.models_fields = {fld.name: fld for fld in model_cls._meta.get_fields()}
        self.describe()
        timestamp = datetime.today().strftime('%Y%m%d%H%M%S')
        self.lines.append(f'\n\nRendered on {timestamp}\n')

    def describe(self):
        number = 0.0
        self.lines.append(f'# Forms')
        self.lines.append(f'## {self.model_cls._meta.verbose_name}')
        self.lines.append(self.model_cls.__doc__)

        for fieldset_name, fields in self.admin_cls.fieldsets:
            if fieldset_name not in ['Audit']:
                fieldset_name = fieldset_name or 'Main'
                self.lines.append(f'\n<b>Section: {fieldset_name}</b>')
                for fnames in fields.values():
                    for fname in fnames:
                        if fname not in DEFAULT_BASE_FIELDS:
                            number = self.get_next_number(number, fname)
                            self.add_field(fname=fname, number=number)


    def add_foreign_keys(self):
        self.lines.append(f'\n<b>Foreign keys:</b>')

    def add_m2ms(self):
        self.lines.append(f'\n<b>Many2Many keys:</b>')
        
    def add_hidden_fields(self):
        self.lines.append(f'\n<b>Hidden fields:</b>')
        self.add_field(fname='report_datetime')
        base_fields = DEFAULT_BASE_FIELDS
        base_fields.sort()
        for fname in base_fields:
            self.add_field(fname=fname)
            
    def add_field(self, fname=None, number=None):
        number = number or '@'
        field_cls = self.models_fields.get(fname)
        self.lines.append(f'\n<b>{number}.</b> {field_cls.verbose_name}')
        if field_cls.help_text:
            self.lines.append(f'\n&nbsp;&nbsp;&nbsp;&nbsp;<i>{field_cls.help_text}</i>')
        self.lines.append(f'* db_table: {db_table}')
        self.lines.append(f'* column: {field_cls.name}')
        self.lines.append(f'* column_type: {field_cls.get_internal_type()}')
        if field_cls.max_length: 
            self.lines.append(f'* length: {field_cls.max_length}')
        if field_cls.get_internal_type() == 'DateField':
            self.lines.append(f'* format: YYYY-MM-DD')
        if field_cls.get_internal_type() == 'DateTimeField':
            self.lines.append(f'* format: YYYY-MM-DD HH:MM:SS.sss (tz=UTC)')
        self.add_field_responses(field_cls=field_cls)
        
    def add_field_responses(self, field_cls=None):
        if field_cls.get_internal_type() == 'CharField':
            if field_cls.choices:
                self.lines.append(f'* responses:')
                for response in [f'<u>{tpl[0]}</u>: <i>{tpl[1]}</i>' for tpl in field_cls.choices]:
                    self.lines.append(f'  - {response} ')
            else:
                self.lines.append('* responses: <i>free text</i>')
        elif field_cls.get_internal_type() == 'ManyToManyField':
            self.lines.append('* responses: <i>Select all that apply</i>')
            for obj in field_cls.related_model.objects.all().order_by('display_index'):
                self.lines.append(f'  - <u>{obj.short_name}</u>: <i>{obj.name}</i> ')
    
    def get_next_number(self, number=None, fname=None):
        if '_other' in fname:
            number += 0.1
        else:
            number = floor(number)
            number += 1.0
        return number




In [141]:
describer = FormDescriber(admin_cls=HivCareAdherenceAdmin)

text = '\n'.join(describer.lines)
# display(Markdown(text))
filename = f'forms_latest.md'
with open(filename, 'w') as f:
    f.write(text)

In [142]:
# add rules related to this model
# rules that this form triggers

# rules that target this form


In [143]:
# validation
